# Imports

In [138]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from IPython.display import clear_output
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import kpss
from scipy.stats import chi2

import plotly.express as px
import plotly.graph_objects as go

In [9]:
df = pd.read_csv('RGPDIC.csv')
df

,DATE,DI
0,1947-01-01,218.091
1,1947-04-01,201.386
2,1947-07-01,195.461
3,1947-10-01,233.284
4,1948-01-01,257.220
...,...,...
300,2022-01-01,3892.480
301,2022-04-01,3746.973
302,2022-07-01,3653.850
303,2022-10-01,3694.128


# Questions 1-9 Trends

## Question 1 (1 point)

Import the time series and plot the data. Is a deterministic trend present in the data?

In [16]:
fig = px.line(df, x='DATE', y='DI')
fig.update_layout(title='Real Gross Private Domestic Investment')
fig.show()

## Question 2 (2 points)

Apply the Augmented Dickey-Fuller (ADF) test 

to examine the presence of a unit root in yt. 

Include in the test equation an intercept term 

and select the number of included lags based on AIC. 

Is a stochastic trend present in the data?

In [21]:
# apply ADF test with an intercept term and select the number of lags based on AIC
result = adfuller(df['DI'], regression='c', autolag='AIC')

# extract and print the test statistics and p-value
test_statistic = result[0]
p_value = result[1]

if p_value < 0.05:
    print(f'Test statistic: {test_statistic:.4f}')
    print(f'p-value: {p_value:.4f}')
    print('The ADF test result suggests that the time series is stationary.')
else:
    print(f'Test statistic: {test_statistic:.4f}')
    print(f'p-value: {p_value:.4f}')
    print('The ADF test result suggests that the time series is non-stationary.')


Test statistic: 0.8118
p-value: 0.9918
The ADF test result suggests that the time series is non-stationary.


## Question 3 (2 points)

Apply the ADF test to examine the presence of a unit root in yt 

including additionally a deterministic trend in the test equation. 

Compare the test results to the results obtained in (2.). 

Given the properties of the time series, 

which specification of the test regression is preferable 

(which deterministic components do you include in the test regression)?

In [22]:
result_ct = adfuller(df['DI'], regression='ct')

test_statistic_ct = result_ct[0]
p_value_ct = result_ct[1]

if p_value_ct < 0.05:
    print(f'Test statistic with trend: {test_statistic_ct:.4f}')
    print(f'p-value with trend: {p_value_ct:.4f}')
    print('The ADF test result suggests that the time series is stationary with a deterministic trend.')
else:
    print(f'Test statistic with trend: {test_statistic_ct:.4f}')
    print(f'p-value with trend: {p_value_ct:.4f}')
    print('The ADF test result suggests that the time series is non-stationary with a deterministic trend.')


Test statistic with trend: -1.9835
p-value with trend: 0.6104
The ADF test result suggests that the time series is non-stationary with a deterministic trend.


## Question 4 (2 points)

Estimate AR(p) models for p = 1, . . . , 12 

for yt including an intercept and deterministic trend. 

Which model does the AIC indicate as the preferred choice?

In [45]:
# Define the range of p values
ps = range(1, 13)

# Initialize AIC list
aics = []

# Iterate over the range of p values and fit ARIMA models
for p in ps:
    model = ARIMA(df['DI'], order=(p, 0, 0), trend='ct')
    result = model.fit()
    aics.append(result.aic)

# Find the index of the minimum AIC
min_idx = np.argmin(aics)


clear_output()

In [46]:
# Create a scatter plot of AIC values vs. p values
ps_array = np.array(ps)
ps_array

fig = go.Figure()
fig.add_trace(go.Scatter(x=ps_array, y=aics, mode='markers', marker=dict(symbol='circle', size=8),
                         name='AIC'))
# Add a red dot for the minimum AIC value
fig.add_trace(go.Scatter(x=[ps_array[min_idx]], y=[aics[min_idx]], mode='markers', 
                         marker=dict(color='red', size=12, symbol='circle'),
                         name='Minimum AIC'))
# Add axis labels and a title
fig.update_layout(title='AIC vs. AR(p) Model Order', xaxis_title='AR Model Order (p)', yaxis_title='AIC')
fig.show()


## Question 5 (3 points)

Estimate an AR(2) model with intercept and deterministic trend for yt. 

Discuss the estimation output. 

What conclusions regarding stationarity can you draw from the estimated parameters?

In [47]:
model = ARIMA(df['DI'], order=(2, 0, 0), trend='ct')
result = model.fit()
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:                     DI   No. Observations:                  305
Model:                 ARIMA(2, 0, 0)   Log Likelihood               -1712.659
Date:                Wed, 17 May 2023   AIC                           3435.318
Time:                        17:42:40   BIC                           3453.919
Sample:                             0   HQIC                          3442.758
                                - 305                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       -326.3806    219.515     -1.487      0.137    -756.622     103.860
x1            11.7186      1.089     10.763      0.000       9.585      13.853
ar.L1          1.0429      0.024     43.308      0.0

## Question 6 (2 points)

Evaluate the estimated AR(2) model based on the estimated residuals. 

Are the residuals following a White Noise process? 

Test whether they are autocorrelated (using a Ljung-Box test) 

and also whether they follow a normal distribution.

In [48]:
residuals = result.resid

In [49]:
fig = px.line(residuals, title="Residuals Plot")
fig.show()

In [57]:
# acorr_ljungbox test for autocorrelation and plot the p-values, x-axis the lag
lb_test = acorr_ljungbox(residuals, lags=12)
lb_test


,lb_stat,lb_pvalue
1,0.039255,0.842944
2,0.124259,0.939761
3,0.641166,0.886947
4,0.960512,0.915723
5,1.686319,0.890619
6,2.866922,0.825355
7,6.264828,0.509190
8,6.505402,0.590808
9,6.844115,0.653345
10,7.111619,0.714867


In [61]:
# lb_test is df, plot the column lb_pvalue and x axis is lag
fig = px.line(lb_test, x=lb_test.index, y='lb_pvalue', title="Ljung-Box Test")
fig.show()

In [68]:
# print interpretation of the test
if lb_test['lb_pvalue'].values[0] < 0.05:
    print('The Ljung-Box test result suggests that the residuals are autocorrelated.')
else:
    print('The Ljung-Box test result suggests that the residuals are not autocorrelated.')

The Ljung-Box test result suggests that the residuals are not autocorrelated.


In [70]:
# test normailty of residuals using Jarque-Bera test
jb_test = sm.stats.stattools.jarque_bera(residuals)

# print interpretation of the test
if jb_test[1] < 0.05:
    print('The Jarque-Bera test result suggests that the residuals are not normally distributed.')
else:
    print('The Jarque-Bera test result suggests that the residuals are normally distributed.')

The Jarque-Bera test result suggests that the residuals are not normally distributed.


## Question 7 (2 points)

Take the first difference of yt. Denote the new series as ∆yt. 

Plot the time series 

and discuss all key properties of time series data for ∆yt based on visual inspection.

In [71]:
dyt = df['DI'].diff().dropna()
dyt

1      -16.705
2       -5.925
3       37.823
4       23.936
5       15.918
        ...   
300     50.709
301   -145.507
302    -93.123
303     40.278
304   -121.050
Name: DI, Length: 304, dtype: float64

In [76]:
# Create a scatter plot of ∆yt
scatter_fig = px.scatter(dyt, x=dyt.index, y=dyt, title='Scatter plot of ∆yt')
scatter_fig.update_xaxes(title_text='Date')
scatter_fig.update_yaxes(title_text='∆yt')

# Create a histogram of ∆yt
hist_fig = px.histogram(dyt, x=dyt, nbins=30, title='Histogram of ∆yt')
hist_fig.update_xaxes(title_text='∆yt')
hist_fig.update_yaxes(title_text='Frequency')

# Show the plots
scatter_fig.show()
hist_fig.show()


## Question 8 (1 point)

Apply the ADF test to examine the presence of a unit root in the ∆yt. 

Given the properties of the time series, 

which deterministic components do you include in the test regression? 

How do you interpret your test results?

In [78]:
# Perform ADF test on first difference of yt
result = adfuller(dyt, maxlag=None, regression='c')

# Extract the test statistic and p-value from the result
test_statistic = result[0]
p_value = result[1]

# Print the results
print(f"ADF test statistic: {test_statistic:.3f}")
print(f"p-value: {p_value:.3f}")

if p_value < 0.05:
    print('The ADF test result suggests that the time series is stationary.')
else:
    print('The ADF test result suggests that the time series is non-stationary.')

ADF test statistic: -16.373
p-value: 0.000
The ADF test result suggests that the time series is stationary.


## Question 9 (1 point)

Compute a Kwiatkowski-Phillips-Schmidt-Shin (KPSS) test 

for the null hypothesis that yt is level or trend stationary. 

Then, compute the KPSS test for the null hypothesis that ∆yt is level stationary. 

How do you interpret your test results?

In [82]:
### KPSS TREND STATIONARY TEST ###

kpss_result = kpss(df['DI'], regression='ct')
print(f'KPSS test statistic: {kpss_result[0]:.4f}')
print(f'p-value: {kpss_result[1]:.4f}')

# interpret the results
if kpss_result[1] < 0.05:
    print('The KPSS test result suggests that the time series is non-stationary.')
else:
    print('The KPSS test result suggests that the time series is stationary.')

KPSS test statistic: 0.5556
p-value: 0.0100
The KPSS test result suggests that the time series is non-stationary.


/var/folders/df/ss33svv50gq79h_yy76phkq80000gn/T/ipykernel_30671/2821065592.py:3: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




In [83]:
### KPSS LEVEL STATIONARY TEST ###

kpss_result_diff = kpss(dyt, regression='c')
print(f'KPSS test statistic: {kpss_result_diff[0]:.4f}')
print(f'p-value: {kpss_result_diff[1]:.4f}')

# interpret the results
if kpss_result_diff[1] < 0.05:
    print('The KPSS test result suggests that the time series is non-stationary.')
else:
    print('The KPSS test result suggests that the time series is stationary.')

KPSS test statistic: 0.2269
p-value: 0.1000
The KPSS test result suggests that the time series is stationary.


/var/folders/df/ss33svv50gq79h_yy76phkq80000gn/T/ipykernel_30671/1294734364.py:3: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.




# Question 10-12 Seasonality

In [127]:
# load RGDICnoSA.csv file
df_ns = pd.read_csv('RGPDICnoSA.csv')
df_ns

,DATE,DIS
0,2003-04-01,557.658
1,2003-07-01,599.846
2,2003-10-01,588.230
3,2004-01-01,582.144
4,2004-04-01,631.168
...,...,...
75,2022-01-01,935.515
76,2022-04-01,930.582
77,2022-07-01,950.800
78,2022-10-01,915.562


## Question 10 (2 points)

The CSV file RGPDICnoSA contains quarterly observations 

on real gross private domestic investment for the United States 

over the period Q2 2003 until Q1 2023 

which are not seasonally adjusted. We denote the series by yt∗.

Import the new time series and plot the data. 

Does this time series exhibit a seasonal pattern? 

Regress yt∗ on a time trend and four seasonal dummy variables. 

Discuss the regression output.

In [128]:
# Plot the data
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_ns['DATE'], y=df_ns['DIS'], mode='lines', name='yt*'))
fig.update_layout(title='Real Gross Private Domestic Investment',
                  xaxis_title='Date', yaxis_title='Value')
fig.show()

In [129]:
# Create the time trend variable
df_ns['Trend'] = range(1, len(df_ns) + 1)

# Create the seasonal dummy variables
seasonal_dummies = pd.get_dummies(df_ns['DATE'].apply(lambda x: pd.Period(x, freq='Q')).apply(lambda x: x.quarter), prefix='Season')
seasonal_dummies = seasonal_dummies.astype(int)

# Concatenate the original DataFrame with the trend and seasonal dummies
df_regress = pd.concat([df_ns, seasonal_dummies], axis=1)

df_regress

,DATE,DIS,Trend,Season_1,Season_2,Season_3,Season_4
0,2003-04-01,557.658,1,0,1,0,0
1,2003-07-01,599.846,2,0,0,1,0
2,2003-10-01,588.230,3,0,0,0,1
3,2004-01-01,582.144,4,1,0,0,0
4,2004-04-01,631.168,5,0,1,0,0
...,...,...,...,...,...,...,...
75,2022-01-01,935.515,76,1,0,0,0
76,2022-04-01,930.582,77,0,1,0,0
77,2022-07-01,950.800,78,0,0,1,0
78,2022-10-01,915.562,79,0,0,0,1


In [130]:
# Perform the regression
X = df_regress[['Trend'] + list(seasonal_dummies.columns)]
y = df_regress['DIS']
model = sm.OLS(y, X)
results = model.fit()

# Print the regression output
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    DIS   R-squared:                       0.696
Model:                            OLS   Adj. R-squared:                  0.679
Method:                 Least Squares   F-statistic:                     42.85
Date:                Thu, 18 May 2023   Prob (F-statistic):           1.15e-18
Time:                        20:59:16   Log-Likelihood:                -452.64
No. Observations:                  80   AIC:                             915.3
Df Residuals:                      75   BIC:                             927.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Trend          4.4617      0.347     12.852      0.0

## Question 11 (2 points)

Estimate an AR(4) model including an intercept and deterministic trend for yt∗. 

Compute the fitted values obtained from the AR(4) model. 

How accurate are the fitted values compared to the actual data yt∗? 

You can answer this by inspecting a plot of the fitted values and actual data.


In [131]:
# add first lag till fourth lag till df_ns
df_ns['DIS_lag1'] = df_ns['DIS'].shift(1)
df_ns['DIS_lag2'] = df_ns['DIS'].shift(2)
df_ns['DIS_lag3'] = df_ns['DIS'].shift(3)
df_ns['DIS_lag4'] = df_ns['DIS'].shift(4)

# drop the first 4 rows
df_ns = df_ns.dropna()

# Perform the regression with trend
X = df_ns[['DIS_lag1', 'DIS_lag2', 'DIS_lag3', 'DIS_lag4', 'Trend']]
X = sm.add_constant(X)
y = df_ns['DIS']
model = sm.OLS(y, X)
results = model.fit()

# Compute the fitted values
fitted_values = results.fittedvalues


In [132]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_ns['DATE'], y=y, mode='lines', name='Actual Data'))
fig.add_trace(go.Scatter(x=df_ns['DATE'], y=fitted_values, mode='lines', name='Fitted Values'))
fig.update_layout(title='AR(4) Model Fitted Values vs Actual Data',
                  xaxis_title='Date', yaxis_title='Value')
fig.show()

## Question 12 (3 points)

Compare the out-of sample forecasting performance of the AR(4) model 

for yt∗ to a random walk model with a deterministic trend 

based on the mean squared forecast errors. 

Compute one-step ahead forecast for the last 50 observations based on a rolling window. 

Test whether the difference in forecast performance of the two models is statistically significant. 

Which of the models is preferable in terms of forecasting accuracy?

In [136]:
# Set the window size
window_size = len(df_ns) - 50

# Initialize lists to store the forecasts
forecasts_ar4 = []
forecasts_rw = []

# Iterate over the last 50 observations
for i in range(window_size, len(df_ns)):
    # Extract the current window
    window = df_ns[:i]

    # Fit the ARIMA(4, 0, 0) model on the window
    arima_model = ARIMA(window['DIS'], order=(4, 0, 0))
    arima_results = arima_model.fit()

    # Compute the forecast for the next observation using ARIMA(4, 0, 0)
    forecast_ar4 = arima_results.forecast(steps=1)
    forecasts_ar4.append(forecast_ar4)

    # Compute the forecast for the next observation using random walk with a deterministic trend
    forecast_rw = window['DIS'].iloc[-1] + window['Trend'].iloc[-1]
    forecasts_rw.append(forecast_rw)

# clear output
clear_output()

# Print the forecasts
print("ARIMA(4, 0, 0) Forecasts:", forecasts_ar4)
print("Random Walk Forecasts:", forecasts_rw)


ARIMA(4, 0, 0) Forecasts: [26    614.175931
dtype: float64, 27    552.437557
dtype: float64, 28    587.271837
dtype: float64, 29    574.350426
dtype: float64, 30    631.586014
dtype: float64, 31    599.969733
dtype: float64, 32    650.05567
dtype: float64, 33    636.201138
dtype: float64, 34    704.293478
dtype: float64, 35    615.422361
dtype: float64, 36    667.69355
dtype: float64, 37    667.242827
dtype: float64, 38    745.653014
dtype: float64, 39    683.615807
dtype: float64, 40    685.532367
dtype: float64, 41    719.918332
dtype: float64, 42    784.373931
dtype: float64, 43    715.364752
dtype: float64, 44    747.070159
dtype: float64, 45    778.292529
dtype: float64, 46    806.55374
dtype: float64, 47    739.026841
dtype: float64, 48    752.158372
dtype: float64, 49    773.365216
dtype: float64, 50    787.382338
dtype: float64, 51    752.922316
dtype: float64, 52    737.767511
dtype: float64, 53    812.159357
dtype: float64, 54    824.043161
dtype: float64, 55    789.990213
dt

In [137]:
# calculate the MSE for ARIMA(4,0,0) and random walk
mse_ar4 = mean_squared_error(df_ns['DIS'].iloc[window_size:], forecasts_ar4)
mse_rw = mean_squared_error(df_ns['DIS'].iloc[window_size:], forecasts_rw)

# Print the MSE values
print(f"ARIMA(4, 0, 0) MSE: {mse_ar4:.3f}")
print(f"Random Walk MSE: {mse_rw:.3f}")

ARIMA(4, 0, 0) MSE: 2431.219
Random Walk MSE: 5045.122


In [139]:
# test whether the MSE of ARIMA(4,0,0) is significantly lower than the MSE of random walk
# Compute the test statistic
test_statistic = (mse_rw - mse_ar4) / (mse_ar4 / (len(df_ns) - 8))

# Compute the p-value
p_value = 1 - chi2.cdf(test_statistic, df=4)

# Print the test statistic and the p-value
print(f"Test statistic: {test_statistic:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print("The MSE of ARIMA(4, 0, 0) is significantly lower than the MSE of random walk.")
else:
    print("The MSE of ARIMA(4, 0, 0) is not significantly lower than the MSE of random walk.")

Test statistic: 73.1096
P-value: 0.0000
The MSE of ARIMA(4, 0, 0) is significantly lower than the MSE of random walk.


# Question 13-14 ADF-test

## Question 13 (4 points)

In order to understand the behavior of the ADF test 

better conduct a small simulation to see the dependence of rejection/no-rejection decision 

on the value of the autoregressive parameter in an AR(1) model. 

To do so, run 1000 simulations in which you simulate a time-series with 100 observations 

based on the following data generating process (DGP):

yt = φyt−1 + εt y0 = 0

εt ∼N(0,1).

Repeat the 1000 simulations for three different values of φ. 

Set (1) φ = 0.5, then (2) φ = 0.9, and finally (3) φ = 0.99. 

Hence, you perform 3 times 1000 simulations and in each simulation you generate a time series of length 100 according to the DGP with a specific φ. 

In each simulation draw, calculate the value of the ADF test statistic. 

Select the setting of the ADF test accordingly based on the knowledge of the DGP. Plot the values of the test statistics for all simulation draws in a histogram.

Compare those to the critical values. 

What do you conclude regarding the performance of the ADF test based on the value of
the autoregressive parameter in an AR(1) model?

In [140]:
n_simulations = 1000
n_obs = 100
phi_values = [0.5, 0.9, 0.99]
adf_stats = np.zeros((n_simulations, len(phi_values)))

In [141]:
for sim in range(n_simulations):
    for i, phi in enumerate(phi_values):
        # Generate the time series
        epsilon = np.random.normal(size=n_obs)
        y = np.zeros(n_obs)
        for t in range(1, n_obs):
            y[t] = phi * y[t-1] + epsilon[t]
        
        # Perform the ADF test
        adf_result = adfuller(y, regression='c')
        adf_stats[sim, i] = adf_result[0]

In [142]:
# Create a histogram trace for each phi value
hist_traces = []
for i, phi in enumerate(phi_values):
    hist_trace = go.Histogram(x=adf_stats[:, i], nbinsx=20, opacity=0.7, name=f'φ = {phi}')
    hist_traces.append(hist_trace)

# Create the layout
layout = go.Layout(
    title='Histogram of ADF Test Statistics for AR(1) Simulations',
    xaxis=dict(title='ADF Test Statistic'),
    yaxis=dict(title='Frequency'),
    legend=dict(title='Autoregressive Parameter (φ)'),
)

# Create the figure and add the histogram traces
fig = go.Figure(data=hist_traces, layout=layout)

# Show the figure
fig.show()

## Question 14 (4 points)

Repeat the simulation for a model with a random walk. 

Run again 1000 simulations generating data from the following DGP

yt = yt−1 + εt y0 = 0

εt ∼N(0,1).

In each simulation draw, calculate the value of the ADF test statistic. 

Select the setting of the ADF test accordingly based on the knowledge of the DGP. 

Plot the values of the test statistics in a histogram. 

Compare those to the critical values. What do you conclude regarding the performance of the ADF test? 

Compare you findings to those of the question before.

In [143]:
n_simulations = 1000
n_obs = 100
adf_stats_rw = np.zeros(n_simulations)

for sim in range(n_simulations):
    # Generate the time series
    epsilon = np.random.normal(size=n_obs)
    y = np.zeros(n_obs)
    for t in range(1, n_obs):
        y[t] = y[t-1] + epsilon[t]
        
    # Perform the ADF test
    adf_result = adfuller(y, regression='c')
    adf_stats_rw[sim] = adf_result[0]

In [144]:
# Create a histogram trace for random walk simulations
hist_trace_rw = go.Histogram(x=adf_stats_rw, nbinsx=20, opacity=0.7)

# Create the layout
layout_rw = go.Layout(
    title='Histogram of ADF Test Statistics for Random Walk Simulations',
    xaxis=dict(title='ADF Test Statistic'),
    yaxis=dict(title='Frequency'),
)

# Create the figure and add the histogram trace
fig_rw = go.Figure(data=hist_trace_rw, layout=layout_rw)

# Show the figure
fig_rw.show()